In [8]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]
results_dict = {}
for r in results: # S
    model_name, values = r[0], r[1:]
    if model_name in results_dict:
        results_dict[model_name].append([values])
    else:
        results_dict[model_name] = [values]

results_dict
    

{'2023_10_17-13_17_30-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean': [['RSICD-CLS',
   'zeroshot-val-top1',
   '70.02'],
  [['UCM-CLS', 'zeroshot-val-top1', '76.05']],
  [['RSICD', 'image_to_text_R@1', '5.21']],
  [['RSICD', 'image_to_text_R@5', '19.74']],
  [['RSICD', 'image_to_text_R@10', '33.82']],
  [['RSICD', 'text_to_image_R@1', '6.03']],
  [['RSICD', 'text_to_image_R@5', '20.84']],
  [['RSICD', 'text_to_image_R@10', '32.72']],
  [['UCM', 'image_to_text_R@1', '4.52']],
  [['UCM', 'image_to_text_R@5', '15.83']],
  [['UCM', 'image_to_text_R@10', '26.73']],
  [['UCM', 'text_to_image_R@1', '4.23']],
  [['UCM', 'text_to_image_R@5', '14.64']],
  [['UCM', 'text_to_image_R@10', '25.06']],
  [['Sydney', 'image_to_text_R@1', '3.62']],
  [['Sydney', 'image_to_text_R@5', '14.29']],
  [['Sydney', 'image_to_text_R@10', '24.14']],
  [['Sydney', 'text_to_image_R@1', '4.83']],
  [['Sydney', 'text_to_image_R@5', '17.10']],
  [['Sydney', 'text_to_image_R@10', '28.57']]],